In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai


In [30]:
qdrant_client = QdrantClient(
    url="http://localhost:6333"
)

qdrant_client.recreate_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

/var/folders/pm/3fb4skk5051dz8cqyfx0qvw80000gp/T/ipykernel_80620/774444838.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [13]:
df_items = pd.read_json("data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

df_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN


In [14]:
def preprocess_data(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [15]:
df_items['preprocessed_title'] = df_items.apply(preprocess_data, axis=1)
df_items['preprocessed_title'].head(2)

0    RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...
1    SNESH-2 Pack USB-C Female to USB Male Adapter,...
Name: preprocessed_title, dtype: object

In [16]:
df_sample = df_items.sample(n=50, random_state=42)
df_sample.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,preprocessed_title
521,All Electronics,"KEEPRO Pencil 2nd Generation for iPad, Magneti...",4.3,2131,[[Compatibility]- ONLY compatible with iPad mi...,[],22.97,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Works great', 'url': 'https://www....",KEEPRO,"[Electronics, Computers & Accessories, Tablet ...",{'Product Dimensions': '6.5 x 0.35 x 0.35 inch...,B0BF18F6R7,NaN,NaN,NaN,"KEEPRO Pencil 2nd Generation for iPad, Magneti..."
737,Industrial & Scientific,HOSONGIN 1/4 TRS Stereo Jack to Dual 1/4 TS Mo...,4.6,114,[EASILY CONNECT DEVICES: HOSONGIN 1/4 inch TRS...,[],14.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'YPP-136 TRS to TS 1/4"" Stereo Brea...",HOSONGIN,"[Electronics, Home Audio, Home Audio Accessori...","{'Brand': 'HOSONGIN', 'Connector Type': 'Auxil...",B0B96LV4C5,NaN,NaN,NaN,HOSONGIN 1/4 TRS Stereo Jack to Dual 1/4 TS Mo...


In [35]:
def get_embeddings(text, model='text-embedding-3-small'):
    response = openai.embeddings.create(input=text, model=model)
    return response.data[0].embedding


get_embeddings("Hello, world!")

[-0.019153675064444542,
 -0.025300605222582817,
 -0.0017183159943670034,
 0.018790507689118385,
 -0.033814843744039536,
 -0.019718600437045097,
 -0.021009858697652817,
 0.05159655585885048,
 -0.03214696794748306,
 -0.030371487140655518,
 -0.0021487355697900057,
 -0.029026426374912262,
 -0.002500132890418172,
 -0.03150133788585663,
 0.010336796753108501,
 0.018548397347331047,
 -0.04616250842809677,
 0.04145479202270508,
 0.0004346229834482074,
 0.041185781359672546,
 0.05369485169649124,
 0.0018713166937232018,
 0.00457657128572464,
 0.00995345413684845,
 0.04780348390340805,
 0.0021756368223577738,
 -0.009792047552764416,
 0.03846875578165054,
 0.0008953065262176096,
 -0.052080780267715454,
 0.0511123351752758,
 -0.03252358362078667,
 -0.014055891893804073,
 -0.012609951198101044,
 0.013248855248093605,
 0.018561847507953644,
 0.001591375796124339,
 -0.0008595783147029579,
 -0.012784808874130249,
 -0.029725857079029083,
 -0.004445428028702736,
 -0.015293348580598831,
 0.02566377259790

In [36]:
data_to_embed = df_sample['preprocessed_title'].tolist()

pointStructs = []

for i, text in enumerate(data_to_embed):
    embedding = get_embeddings(text)
    pointStructs.append(PointStruct(id=i, vector=embedding, payload={"text": text}))

In [37]:
pointStructs

[PointStruct(id=0, vector=[0.010869191028177738, -0.014954255893826485, 0.011192244477570057, -0.011265192180871964, -0.05060478672385216, -0.0003106785879936069, -0.038787275552749634, 0.04174686223268509, 0.0026130869518965483, -0.01020745187997818, -0.02903314121067524, 0.014506149105727673, -0.03105483017861843, 0.08687015622854233, 0.01433941163122654, 0.004861435852944851, -0.03586937114596367, 0.009087185375392437, -0.01874753087759018, 0.01695510372519493, 0.010098030790686607, 0.017246894538402557, 0.034014418721199036, 0.03065882995724678, 0.021738382056355476, 0.02112353779375553, -0.03311820700764656, -0.026427870616316795, -0.02307227998971939, 0.05206373706459999, -0.013255619443953037, -0.020269008353352547, -0.019477006047964096, -0.04881235957145691, -0.04114244133234024, -0.01925816386938095, -0.01727815717458725, 0.015516994521021843, -0.018059739843010902, 0.0006089821108616889, 0.02403102070093155, 0.040016964077949524, 0.008065919391810894, 0.000803074799478054, -

In [38]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    points=pointStructs,
    wait=True
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
# WE CAN ALSO QUERY THE DATA
def retrieve_data(query, k=10):
    response = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=get_embeddings(query),
        limit=k
    )
    return response


retrieve_data("what airphones can I get?", k=10).points

[ScoredPoint(id=12, version=0, score=0.46780694, payload={'text': "TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black ♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles are supported. The wireless earbuds are compatible with most Bluetooth enabled iPhones, Andriods, smart TVs, computers, etc. Each wireless earbuds will pair with each other when they are removed from the charging case. From here, enable Bluetooth on your chosen device and pair with the headphones. ♬【Clear Call & Sound quality】Each wireless earbuds features a 10mm diameter speaker and 2 microphones to reduce ambient noise and transmit your voice for a clear call in any environment. For the music enjoyers, these wireless earbuds offer a deep bass for an immersive musical experience. ♬【Long Battery Life &

In [7]:
qdrant_client

In [11]:
### Let's create a dataset

qdrant_client = QdrantClient(
    url="http://localhost:6333"
)
qdrant_client.create_collection(
    collection_name='Amazon-items-collection-02',
    vectors_config=VectorParams(
        size=1536, distance=Distance.COSINE)
    )

True

In [18]:
df_sample = df_items.sample(n=50, random_state=25)

In [19]:
def get_embeddings(text, model='text-embedding-3-small'):
    response = openai.embeddings.create(input=[text], model=model)
    return response.data[0].embedding


get_embeddings("Hello, world!")

[-0.019143931567668915,
 -0.025292053818702698,
 -0.0017211713129654527,
 0.01883450709283352,
 -0.03382139280438423,
 -0.019682060927152634,
 -0.02102738246321678,
 0.05160655081272125,
 -0.03218010067939758,
 -0.03043118305504322,
 -0.0021508336067199707,
 -0.028924422338604927,
 -0.0024871639907360077,
 -0.03148053586483002,
 0.010291713289916515,
 0.01856544241309166,
 -0.04614454507827759,
 0.04140901193022728,
 0.00043050304520875216,
 0.04116685315966606,
 0.053651440888643265,
 0.0018481360748410225,
 0.004564004950225353,
 0.009955382905900478,
 0.04781274124979973,
 0.002164286794140935,
 -0.00984775647521019,
 0.038422394543886185,
 0.0009131372789852321,
 -0.05209086835384369,
 0.051122233271598816,
 -0.032529886811971664,
 -0.01408552099019289,
 -0.012605667114257812,
 0.013271600939333439,
 0.01856544241309166,
 0.0016320437425747514,
 -0.0008479732787236571,
 -0.012773832306265831,
 -0.029677802696824074,
 -0.004510191734880209,
 -0.015309764072299004,
 0.025668743997812

In [21]:
data_to_embed = df_sample['preprocessed_title'].tolist()

pointStructs = []

for i, text in enumerate(data_to_embed):
    embedding = get_embeddings(text)
    pointStructs.append(PointStruct(id=i, vector=embedding, payload={"text": text}))

In [22]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-02",
    points=pointStructs,
    wait=True
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
# Lets create a synthetic data

import json


batch_output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question."
            },
            "chunk_ids": {
                "type": "array",
                "items": {
                    "type": "integer",
                    "description": "Index of the chunk that could be used to answer the question."
                }
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer grounded in the context."
            },
            "reasoning": {
                "type": "string",
                "description": "Reasoning why the question could be answered with the chunks."
            }
        }
    }
}


SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with indexes of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the indexes of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Try to have a mix of questions that could use multiple chunks and questions that could use single chunk.
Also, include 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{json.dumps(batch_output_schema, indent=2)}
</OUTPUT JSON SCHEMA>

I need to be able to parse the json output.
"""


USER_PROMPT = f"""
Here is the list of chunks, each list element is a dictionary with id and text:
{[{"id": i, "text": data} for i, data in enumerate(data_to_embed)]}
"""




In [25]:
print(USER_PROMPT)


Here is the list of chunks, each list element is a dictionary with id and text:
[{'id': 0, 'text': 'Bluetooth Car Adapter, LDNIO Bluetooth FM Transmitter for Car, 43W PD&QC 3.0 Three USB Port Car Bluetooth Adapter with LED Display, Hands-Free Calling, and AUX Input for All Smartphones Audio Player Fast Charging Type C Multi Ports: USB Type C Durable Fast Connect & Clear Bluetooth Sound 3 in 1 Value'}, {'id': 1, 'text': 'Bluetooth Multi-Device Keyboard, Dual Channel Universal Rechargeable Wireless Keyboard with Integrated Stand for iPad Smartphone Tablet MacBook iOS Windows Android Devices - Pink 【Easy-Switch to 2 Devices】 Simply press the FN+BT1/BT2 to switch typing between 2 connected Bluetooth devices, work with your smartphone and tablet on the slot stablely. 【Type Anywhere in Comfort】0.46in thick and 11.34in long, the compact Bluetooth keyboard is small enough to tuck into your briefcase and light enough to hold in hand. Minimalist layout lets you multitask at home or on the go. 【

In [26]:
print(SYSTEM_PROMPT)


I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with indexes of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the indexes of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Try to have a mix of questions that could use multiple chunks and questions that could use single chunk.
Also, include 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "question": {
        "type": "string",
       

In [29]:
response = openai.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)

In [30]:
print(response.choices[0].message.content)

```json
[
  {
    "question": "Which products are available for wireless Bluetooth audio playback?",
    "chunk_ids": [0,2,5,9,23,47,49],
    "answer_example": "We have the following products that support wireless Bluetooth audio playback: Bluetooth Car Adapter (id:0), Active Noise Cancelling Wireless Earbuds (id:2), Glorious Model D Wireless Gaming Mouse (id:5, note: it's a mouse, not an audio device but mentions Bluetooth), maio Bluetooth Speaker (id:9), Tiksounds Wireless Earbuds (id:47), Razer Leviathan V2 X PC Soundbar (id:49), and the co2CREA Soft Silicone Case (id:23, case for Bose Bluetooth speaker - actual speaker sold separately).",
    "reasoning": "These chunks mention Bluetooth-enabled audio devices like speakers, earbuds, and adapters, as well as cases for Bluetooth speakers."
  },
  {
    "question": "Do you have any accessories compatible with iPhone 14?",
    "chunk_ids": [13,22,26,39],
    "answer_example": "Yes, we have several accessories compatible with iPhone 14, 

In [33]:
import json

json_output = response.choices[0].message.content
json_output = json_output.replace("```json", "")
json_output = json_output.replace("```", "")
json_output = json.loads(json_output)

In [34]:
json_output

[{'question': 'Which products are available for wireless Bluetooth audio playback?',
  'chunk_ids': [0, 2, 5, 9, 23, 47, 49],
  'answer_example': "We have the following products that support wireless Bluetooth audio playback: Bluetooth Car Adapter (id:0), Active Noise Cancelling Wireless Earbuds (id:2), Glorious Model D Wireless Gaming Mouse (id:5, note: it's a mouse, not an audio device but mentions Bluetooth), maio Bluetooth Speaker (id:9), Tiksounds Wireless Earbuds (id:47), Razer Leviathan V2 X PC Soundbar (id:49), and the co2CREA Soft Silicone Case (id:23, case for Bose Bluetooth speaker - actual speaker sold separately).",
  'reasoning': 'These chunks mention Bluetooth-enabled audio devices like speakers, earbuds, and adapters, as well as cases for Bluetooth speakers.'},
 {'question': 'Do you have any accessories compatible with iPhone 14?',
  'chunk_ids': [13, 22, 26, 39],
  'answer_example': 'Yes, we have several accessories compatible with iPhone 14, including a CarPlay USB A 

In [37]:
from langsmith import Client
import os

client = Client(api_key=os.environ['LANGSMITH_API_KEY'])
dataset_name = "rag-evaluation-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description='Dataset for evaluating RAG pipeline'
)

In [38]:
for item in json_output:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item["question"]},
        outputs={
            "ground_truth": item["answer_example"],
            "context_ids": item["chunk_ids"],
            "contexts": [
                qdrant_client.retrieve(collection_name="Amazon-items-collection-02", ids=[id], with_payload=True)[0].payload["text"]
                for id in item["chunk_ids"]
            ]
        }
    )